# Healthy Plate Builder

Our BILD 62 final project aims to promote healthier, yet tasty, eating habits. The USA has many unhealthy food options and has oversaturated our daily meals with fast and overprocessed foods. Those options are not nutritional and beneficial for a balanced diet, so using our program would help guide people in a better direction regarding choosing what to eat. It creates interactive solutions to promote a better and mindful diet.

## About this project
By utilizing data from the USDA food database we would like to offer an interactive plate builder where users can build their plate with a selection of foods that are nutritious (fruits, veggies, protein, dairy, carbohydrates). Once a user has finished building their plate we hope to provide a nutrional analysis of the plate with details on balanced the meal is and suggestions for meal improvements.

In [ ]:
# this is where we will have our final project 

### Part 1: Importing Data

This part of our project we aim to complete a couple of things: 1.Succesfully import the data we'd like to use , 2. Clean our data, 3. Have our data ready for use in the following parts.

The data we are using comes from the FoodData Central of the USDA. Definitions for the terms and labels used in our chosen dataset is [here](Bild62/Data/Field_Descriptions.pdf)


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.animation as animation
import json
import os
import sys

# I was going to download all the food data information offered on the 
# website but instead will just start with one until everything works

#foundation_food = pd.read_json("Data/foundationfoods.json")

#ffnormalized = pd.json_normalize(foundation_food['FoundationFoods'])

#ffnormalized.head() #there are dictionaries within the columns so need to figure out a way to extract those


food = pd.read_csv("Data/foundationfoodcsv/food.csv")
food_nutrient = pd.read_csv("Data/foundationfoodcsv/food_nutrient.csv")
nutrient = pd.read_csv("Data/foundationfoodcsv/nutrient.csv")
food_category = pd.read_csv("Data/foundationfoodcsv/food_category.csv")


food = food.merge(food_category, left_on="food_category_id", right_on="id", how="left")


food_nutrient = food_nutrient.merge(nutrient, left_on="nutrient_id", right_on="id", how="left")


combined = food.merge(food_nutrient, left_on="fdc_id", right_on="fdc_id", how="left")


nutrient_focus = combined[combined["name"].isin(["Energy", "Protein", "Carbohydrate, by difference", "Total lipid (fat)"])]

# Pivot nutrients into columns
foodfoundation = nutrient_focus.pivot_table(
    index=["fdc_id", "description_x", "code", "description_y"],  
    columns="name",  
    values="amount",  
    aggfunc="first"  
).reset_index()


foodfoundation.rename(columns={
    "description_x": "food_description",
    "Energy": "Calories",
    "Protein": "Protein (g)",
    "Carbohydrate, by difference": "Carbohydrates (g)",
    "Total lipid (fat)": "Fat (g)"
}, inplace=True)


display(foodfoundation)




/var/folders/yv/b13k2r3j49v3bf8rl0672t040000gn/T/ipykernel_8406/440970165.py:20: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  food_nutrient = pd.read_csv("Data/foundationfoodcsv/food_nutrient.csv")


name,fdc_id,food_description,code,description_y,Carbohydrates (g),Calories,Protein (g),Fat (g)
0,319877,Hummus,1600.0,Legumes and Legume Products,NaN,NaN,NaN,19.00
1,319882,Hummus,1600.0,Legumes and Legume Products,NaN,NaN,NaN,18.70
2,319892,Hummus,1600.0,Legumes and Legume Products,NaN,NaN,NaN,16.60
3,319899,Hummus,1600.0,Legumes and Legume Products,NaN,NaN,NaN,19.10
4,319908,Hummus,1600.0,Legumes and Legume Products,NaN,NaN,NaN,18.20
...,...,...,...,...,...,...,...,...
4506,2714569,"Plum, black, with skin, pitted, raw",900.0,Fruits and Fruit Juices,NaN,NaN,NaN,0.14
4507,2714586,"Plum, black, with skin, pitted, raw",900.0,Fruits and Fruit Juices,NaN,NaN,NaN,0.26
4508,2714607,"Plum, black, with skin, pitted, raw",900.0,Fruits and Fruit Juices,NaN,NaN,NaN,0.41
4509,2714625,"Plum, black, with skin, pitted, raw",900.0,Fruits and Fruit Juices,NaN,NaN,NaN,0.27


In [6]:
print(food.columns)


Index(['fdc_id', 'data_type', 'description_x', 'food_category_id',
       'publication_date', 'id', 'code', 'description_y'],
      dtype='object')


### Part 2: Design our Interactive Aspects

This part of our project we aim to sort out how we would like for user input to work

In [14]:
#I know on the google doc it says users will be able to create a plate from a selection
# but it may be easier to just let people input foods so thats what I'll add for now
# we can move scripts to another file later


#This is establishing a place for users to be able to save their plates.
data_folder = "Data"
user_data_file = os.path.join(data_folder, "user_plates.json")

if not os.path.exists(data_folder):
    os.makedirs(data_folder)

if not os.path.exists(user_data_file):
    with open(user_data_file, 'w') as file:
        json.dump({}, file)

#This function can be used for those who want to save their plates. This was feedback we got on the propsal.

def save_plate(username, password, plate, file_path):
    with open(file_path, 'r') as file:
        user_data = json.load(file)
    user_data[username] = {"password": password, "plate": plate}
    with open(file_path, 'w') as file:
        json.dump(user_data, file, indent=4)
    print(f"Plate saved for user '{username}'!")


#This will be fine for now but need to add options to a.build a plate b.revisit a plate c.other

plate = []
print("Welcome to our Healthy Plate Builder \n")

print("Please add foods to your plate one at a time \n")

print("Type 'stop' when you're done adding foods to your plate.\n")

while True:
    food = input("Enter a food to add to your plate: ").strip()
    if food.lower() == "stop":
        print("Your plate is complete!")
        break
    elif food:
        plate.append(food)
        print(f"Added {food} to your plate.")
    else:
        print("You didn't enter any food. Try again.")

print("\nHere is what's on your plate:")
for item in plate:
    print(f"- {item}")

save_option = input("\nDo you want to save your plate? (yes/no): ").strip().lower()

if save_option == "yes":
    username = input("Enter a username: ").strip()
    password = input("Enter a password: ").strip()
    save_plate(username, password, plate, user_data_file)
elif save_option == "no":
    print("You chose not to save your plate.")
else:
    print("Invalid input. Plate was not saved.")

view_option = input("\nDo you want to view saved plates? (yes/no): ").strip().lower()
if view_option == "yes":
    with open(user_data_file, 'r') as file:
        saved_data = json.load(file)
        print("\nSaved Plates:")
        for user, data in saved_data.items():
            print(f"User: {user}, Plate: {data['plate']}")
else:
    print("Thank you for using the plate builder!")



        

Welcome to our Healthy Plate Builder 

Please add foods to your plate one at a time 

Type 'stop' when you're done adding foods to your plate.



Enter a food to add to your plate:  bacon


Added bacon to your plate.


Enter a food to add to your plate:  beef


Added beef to your plate.


...

Your plate is complete!

Here is what's on your plate:
- bacon
- beef
- cheese



Do you want to save your plate? (yes/no):  yes
Enter a username:  testuser1
Enter a password:  testuser1


Plate saved for user 'testuser1'!



Do you want to view saved plates? (yes/no):  no


Thank you for using the plate builder!


In [15]:
#Going to test retrieving saved plates in this cell

def retrieve_plate(username, password, file_path):
    with open(file_path, 'r') as file:
        user_data = json.load(file)
    if username in user_data:
        if user_data[username]["password"] == password:
            return user_data[username]["plate"]
        else:
            return "Incorrect password."
    else:
        return "Username not found."

retrieve_option = input("Would you like to retrieve your previously created plate? (yes/no):\n").strip().lower()

if retrieve_option == "yes":
    username = input("Enter your username: ").strip()
    password = input("Enter your password: ").strip()
    result = retrieve_plate(username, password, user_data_file)
    if isinstance(result, list):
        print(f"\nHere is your saved plate, {username}:")
        for item in result:
            print(f"- {item}")
    else:
        print(f"\n{result}")
else:
    print("You said no to retrieving your plate.")

Would you like to retrieve your previously created plate? (yes/no):
 yes
Enter your username:  testuser1
Enter your password:  testuser1



Here is your saved plate, testuser1:
- bacon
- beef
- cheese


### Part 3: Nutritional Analysis

This part of our project we aim to do the necessary work that will allow for user input to be translated into nutritional details.

### Part 4: Visualization

This part of our project we aim to neatly display the results of Parts 1 and 2 to our users. 

### Part 5: Healthy Plate Builder in Action

This part of our project is where we'd like for our users to be able to use our interactive plate builder . 